In [1]:
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import multiprocessing as mp

import pickle
import json
import functools
import matplotlib
import itertools
import copy

import esipy as esi

from tqdm.notebook import tqdm, trange

import requests
import time

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')

# Currently, memory growth needs to be the same across GPUs
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
    
logical_gpus = tf.config.experimental.list_logical_devices('GPU')
print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")

1 Physical GPUs, 1 Logical GPUs


In [3]:
import time

class Timer:    
    def __enter__(self):
        self.start = time.clock()
        return self

    def __exit__(self, *args):
        self.end = time.clock()
        self.interval = self.end - self.start
        
        print(self.interval)

In [4]:
# Set up network access
esi_client = esi.EsiClient(
    transport_adapter = requests.adapters.HTTPAdapter(
        pool_connections=100,
        pool_maxsize=100,
        max_retries=10,
        pool_block=False
    )
)
esi_app    = esi.EsiApp()

app = esi_app.get_latest_swagger

def try_request(req):
    while True:
        response = esi_client.request(req)
        
        if response.status == 200:
            return response.data
        
        print(response.status)
        print(response.header)
        time.sleep(5)

Defining a 'User-Agent' header is a good practice, and allows CCP to contact you if required. To do this, simply add the following when creating the client: headers={'User-Agent':'something'}.
d:\programme\python 3.6.6\lib\site-packages\esipy\client.py:82: UserWarning: Defining a 'User-Agent' header is a good practice, and allows CCP to contact you if required. To do this, simply add the following when creating the client: headers={'User-Agent':'something'}.
  warnings.warn(warning_message)


In [5]:
# Load universe data
def intd(f):
    return {int(k) : v for k,v in json.load(f).items()};

with open('universe/systems.txt') as f:
    systems = intd(f);

with open('universe/stargates.txt') as f:
    stargates = intd(f);
    
with open('universe/market_types.txt') as f:
    types = intd(f)
    
with open('universe/constellations.txt') as f:
    constellations = intd(f)

In [6]:
# Construct derived data about the universe
systems_graph = nx.Graph();

for system in systems:
    systems_graph.add_node(system);

for stargate in stargates.values():
    systems_graph.add_edge(stargate["system_id"], stargate["destination"]["system_id"])

root = 'Jita'
root_id = [v for v in systems if systems[v]["name"] == root][0]

# Limit to high-sec systems
subselect = [k for k in systems if systems[k]["security_status"] >= 0.5]
subgraph = systems_graph.subgraph(subselect)

# Limit to component connected to Jita
subselect = nx.node_connected_component(subgraph, root_id)
subgraph = systems_graph.subgraph(subselect)
print('High-sec has {} systems'.format(len(subgraph)))

system_ids = list(subgraph)
type_ids = list(types)

landmark_names = ['Jita', 'Amarr', 'Rens', 'Hek', 'Dodixie', 'Oursulaert', 'Tash-Murkon Prime', 'Agil'];
landmarks = [v["system_id"] for v in systems.values() if v["name"] in landmark_names]

# Compute pair-wise distances
print('Computing system distances')
distances = dict(tqdm(nx.shortest_path_length(subgraph), total = len(subgraph)))

@functools.lru_cache(maxsize=None)
def system_distance(s1, s2):
    #return nx.shortest_path_length(subgraph, s1, s2)
    return distances[s1][s2]

@functools.lru_cache(maxsize=None)
def get_region_id(system_id):
    return constellations[systems[system_id]["constellation_id"]]["region_id"]

@functools.lru_cache(maxsize=None)
def get_station(station_id):
    esi_op = app.op['get_universe_stations_station_id'](station_id = station_id);
    return esi_client.request(esi_op).data

@functools.lru_cache(maxsize=None)
def in_highsec(station_id):
    try:
        return get_station(station_id).system_id in subgraph
    except:
        return False

print('Loading regions')
region_ids = list(set(
    get_region_id(system_id) for system_id in subgraph
))

High-sec has 972 systems
Computing system distances



Loading regions


In [7]:
# Load market orders from csv file
orders = pd.read_csv('market/current.csv', index_col = 'order_id')
orders['region_id'] = orders['system_id'].apply(get_region_id)
orders.sort_index(inplace = True)

In [77]:
## Structure of the state space
class StateProp:
    def __init__(self, default, convert = (lambda x: x, lambda x: x)):
        self.convert_in = convert[0]
        self.convert_out = convert[1]
        self.default = default

def to_fixed(x):
    if x is None:
        return None
    return State(x)

def to_mutable(x):
    if x is None:
        return none
    return MutableState(x)

# Dict props should be stored as (key, value) tuples
dict_prop = (lambda x: tuple((i for i in x.items())), lambda x: {k:v for (k,v) in x})

state_props = {
    # Position of the ship
    'system' : StateProp(0),
    'station' : StateProp(None),
    
    # Configuration of the ship
    'volume_limit' : StateProp(0),
    'collateral_limit' : StateProp(0),
    
    # Resource balance
    'time_left' : StateProp(0),
    'wallet' : StateProp(0),
    'cargo' : StateProp({}, dict_prop),
    
    # Market cursors
    'market_group' : StateProp(None),
    'market_item' : StateProp(None),
    'last_modified_item' : StateProp(None),
    'updated_orders' : StateProp({}, dict_prop)
}
    
# Implementation of the state space
class State:
    def __init__(self, inp):
        for name, prop in state_props.items():
            inval = getattr(inp, name, prop.default)
            inval = prop.convert_in(inval)
            object.__setattr__(self, '_prop_' + name, inval)
            
        self._encode_cache = None
    
    def __getattr__(self, name):
        if name in state_props:
            return getattr(self, '_prop_' + name)
        else:
            raise AttributeError('Unknown attribute ' + name)
    
    def __setattr__(self, name, val):
        if name in state_props:
            raise AttributeError('Attribute {} is immutable'.format(name))
        
        object.__setattr__(self, name, val)
    
    def __hash__(self):
        h = 0
        
        for name in state_props:
            h = h ^ hash(getattr(self, name))
        
        return h
    
    def __eq__(self, other):
        for name in state_props:
            if getattr(self, name) != getattr(other, name):
                return False
        
        return True
    
    def __repr__(self):
        (vleft, cleft) = self.limits_left()
        vmax = self.volume_limit
        cmax = self.collateral_limit
        v = vmax - vleft
        c = cmax - cleft
        
        orders_by_id = orders.loc[[k for k,v in self.updated_orders]]
        orders_by_id['vol_mod'] = [v for k,v in self.updated_orders]
        
        cargo_df = pd.DataFrame.from_dict(dict(self.cargo), orient = 'index', columns = ['Amount', 'Value'])
        cargo_df.index.name = 'Item ID'
        cargo_df['Item name'] = cargo_df.index.map(lambda x : types[x]["name"])
        cargo_df = cargo_df[['Item name', 'Amount', 'Value']]
        
        def cargo_entry_info(entry):
            item_id, (amount, value) = entry
            
            item_name = types[item_id]["name"]
            
            return "{:>10} x {:<20} ({:>10.2f} ISK)".format(amount, item_name, value)
        
        def indent(x, n):
            return x.replace("\n", "\n" + " " * n)
        
        cargo_string = "\n".join([cargo_entry_info(e) for e in self.cargo])
        
        try:
            self.validate()
            validated = 'Valid'
        except Exception as e:
            validated = str(e)
            
        return """
State:
    Validation: {valid}
    
    System:     {system}
    Station ID: {station}
    
    Resources:
        Time:       {t:>10.2f}
        Wallet:     {wallet:>10.2f}
        Collateral: {cleft:>10.2f} / {cmax:>10.2f} ({c:>10.2f} used)
        Cargo:      {vleft:>10.2f} / {vmax:>10.2f} ({v:>10.2f} used)
        
    Cargo contents:
        {cargo}
        
    Modified orders:
        {orders}
        """.format(
            valid = validated,
            system = systems[self.system]["name"] if self.system is not None and self.system in systems else "Unknown (ID = {})".format(self.system),
            station = self.station,
            
            t = self.time_left,
            wallet = self.wallet,
            c = c, cmax = cmax, cleft = cleft,
            v = v, vmax = vmax, vleft = vleft,
            
            cargo = indent(cargo_string, 8),
            #cargo  = indent(cargo_df.to_string(), 8),
            orders = indent(orders_by_id.to_string(), 8)
        )
    
    def limits_left(self):
        total_vol = 0
        total_col = 0
        
        for item, (count, value) in self.cargo:
            total_vol += types[item]["volume"] * count
            total_col += value
        
        return (self.volume_limit - total_vol, self.collateral_limit - total_col)
    
    def validate(self):
        assert self.system is not None, 'No system specified'
        assert self.system in system_ids, 'Not in a known system'
        
        (v, c) = self.limits_left()
        assert v >= 0, 'Volume exceeded'
        assert c >= 0, 'Collateral exceeded'
        
        assert self.time_left >= 0, 'Out of time'
    
    @property
    def value(self):
        return self.wallet
        

# Helper class for incremental state modification
class MutableState:
    def __init__(self, other = None):
        if other is None:
            for name, prop in state_props.items():
                setattr(self, name, copy.copy(prop.default))
        else:
            for name, prop in state_props.items():
                setattr(self, name, prop.convert_out(getattr(other, name)))

In [78]:
# Tests involving state manipulation

s = MutableState()
s.system = root_id
s.updated_orders = {
    911191013 : 200
}
s.cargo = {
    34 : (5, 200),
    36 : (10000, 800)
}

s.volume_limit = 10000
s.collateral_limit = 10000
s.time_left = -1

s = State(s)

print(s)


State:
    Validation: Out of time
    
    System:     Jita
    Station ID: None
    
    Resources:
        Time:            -1.00
        Wallet:           0.00
        Collateral:    9000.00 /   10000.00 (   1000.00 used)
        Cargo:         9899.95 /   10000.00 (    100.05 used)
        
    Cargo contents:
                 5 x Tritanium            (    200.00 ISK)
             10000 x Mexallon             (    800.00 ISK)
        
    Modified orders:
                   duration  is_buy_order                     issued  location_id  min_volume  order_id.1  price   range  system_id  type_id  volume_remain  volume_total  region_id  vol_mod
        order_id                                                                                                                                                                             
        911191013       365         False  2020-02-28T12:33:48+00:00     60000061           1   911191013   14.0  region   30003084       41          2357

In [ ]:
# --- Action calculus ---

# Wrapper that allows an action to just modify a mutable state representation
# and performs validity checking of resulting state
class ActionImpl:
    def __init__(self, f, desc = None, repr = None):
        self.wrapped = f
        self.desc = desc
        self.repr = desc if repr is None else repr
    
    def __call__(self, state):
        s = MutableState(state)
        s.last_modified_item = None
        self.wrapped(s)
        s = State(s)
        
        s.validate()
        
        return s
    
    def __repr__(self):
        return self.repr
    
    def __str__(self):
        return self.desc
        
def action(**kwargs):
    def impl(f):
        return ActionImpl(f, **kwargs)
    return impl

In [9]:
# --- Marker broker implementation and market actions ---

def market_transaction(state, type_id, amount, order_type):
    assert order_type in ['buy', 'sell']
    
    # For the duration of this function, we need this element to exist
    if type_id not in state.cargo:
        state.cargo[type_id] = (0, 0)
    
    # Get all possible orders in appropriate order
    candidate_orders = orders[
        (orders.location_id == state.station) &
        (orders.type_id == type_id) &
        (orders.is_buy_order == (order_type == 'buy'))
    ].sort_values('price', ascending = order_type == 'buy')
    
    # Check that we only sell as many items as we have
    if order_type == 'sell':
        amount = min(amount, state.cargo[type_id][0])    
    
    factor = 1 if order_type == 'buy' else -1
    
    for order in candidate_orders.itertuples():
        # Check if we even need anything more
        if amount == 0:
            break
        
        # Check if we can get something from this order
        remaining = order.volume_remain
        
        if order.order_id in state.updated_orders:
            remaining = state.updated_orders[order.order_id]
                
        transfer = min(amount, remaining)
        
        # Don't buy more than we can afford or store on our ship
        if order_type == 'buy':
            transfer = min(
                transfer,
                int(state.wallet / order.price)
            )
    
            vol_left, coll_left = state.limits_left()
            can_store = int(min(
                vol_left / types[type_id].volume,
                coll_left / order.price
            ))
            
            transfer = min(
                transfer,
                can_store
            )
        
        if transfer == 0:
            continue
        
        # Conduct transaction
        delta = transfer if order_type == 'buy' else -transfer
        state.wallet -= delta * order.price
        
        (cnum, cval) = state.cargo[type_id]
        state.cargo[type_id] = (
            cnum + delta,
            cval + delta * order.price
        )
        
        # Record that the order was partially filled
        state.updated_orders[order.order_id] = remaining - transfer
        
        # Continue buying if neccessary
        amount -= transfer
    
    # Remove empty cargo categories
    if state.cargo[type_id][0] == 0:
        del state.cargo[type_id]
    
def buy(type_id, amount):
    @action(desc = 'Buy {} of {}'.format(amount, types[type_id]["name"]))
    def do_buy(s):
        market_transaction(s, type_id, amount, 'buy')
        
    return do_buy

def sell(type_id, amount):
    @action(desc = 'Sell {} of {}'.format(amount, types[type_id]["name"]))
    def do_sell(s):
        market_transaction(s, type_id, amount, 'sell')
    
    return do_sell

In [10]:
# --- Movement actions ---

warp_cost = 1.0

@functools.lru_cache(maxsize=None)
def warp_to_station(station_id):
    station = get_station(station_id)
    
    distances = nx.shortest_path_length(subgraph, station.system_id)
    
    @action(desc = 'Move to {} in {}'.format(station.name, systems[station.system_id]["name"]))
    def do_warp_to_station(s):
        assert s.station != station_id, 'Cannot warp from to same station'
        
        if s.system == station.system_id:
            t = warp_cost
        else:
            #t = warp_time * nx.shortest_path_length(subgraph, s.system, station.system_id)
            t = warp_cost * distances[s.system]
        
        s.station = station_id
        s.system = station.system_id
        
        s.time_left -= t
    
    return do_warp_to

@functools.lru_cache(maxsize=None)
def warp_to_system(system_id):
    distances = nx.shortest_path_length(subgraph, system_id)
    
    @action(desc = 'Move to system {}'.format(systems[system_id]["name"]))
    def do_warp_to_system(s):
        t = warp_cost * system_distance(system_id, s.system)
        
        s.system = system_id
        s.station = None
        
        s.time_left -= t
    
    return do_warp_to_system

In [79]:
# --- Encoding of table data into TF tensors ---

@functools.lru_cache(maxsize=None)
def encode_system(system_id, state = None):
    assert system_id in system_ids
    
    index = system_ids.index(system_id)
    system = systems[system_id]
    
    return [index, system["security_status"]] + [system_distance(l, system_id) for l in landmarks]

@functools.lru_cache(maxsize=None)
def encode_type(type_id, state = None):
    assert type_id in type_ids
    
    index = type_ids.index(type_id)
    t = types[type_id]
    
    return [
        index,
        t["volume"]
    ]

# Encoding of a single order
def encode_order(order, state = None):
    tdata = encode_type(order.type_id, state)
    sdata = encode_system(order.system_id, state)
    
    if tdata is None or sdata is None:
        return None
    
    remaining = order.volume_remain
    
    if state is not None:
        updates = {k:v for k,v in state.updated_orders}
        
        if order.order_id in updates:
            remaining = updates[order.order_id]
    
    return [tdata[0], sdata[0]] + tdata[1:] + sdata[1:] + [
        order.location_id,
        remaining,
        1 if order.is_buy_order else 0,
        0 if order.is_buy_order else 1
    ]

# Encodes an order dataframe into a ragged tensor and an object that can be used to apply deltas to that tensor
def encode_orders(orders):
    # Identify all orders that have valid system and type ids
    candidates = orders[
        (orders.system_id.isin(system_ids)) &
        (orders.type_id.isin(type_ids))
    ].reset_index().set_index(['region_id', 'type_id']).sort_index()

    # Compute the no. of counts per group
    counts = candidates.order_id.groupby(['region_id', 'type_id']).count()
    
    # Fill empty counts with 0
    new_idx = pd.MultiIndex.from_product(
        [region_ids, types],
        names = counts.index.names
    )
    counts = counts.reindex(new_idx, fill_value = 0)
    
    # Encode the identified orders into a tensor
    print('Encoding orders & transferring to GPU')
    flat_data = tf.constant([
        encode_order(o)
        for o in tqdm(list(
            candidates.reset_index().itertuples()
        ))
    ])
    
    # Create a ragged tensor from the uniform tensor
    result = tf.RaggedTensor.from_uniform_row_length(
        tf.RaggedTensor.from_uniform_row_length(
            tf.RaggedTensor.from_row_lengths(
                flat_data,
                counts.to_numpy()
            ),
            len(type_ids)
        ),
        len(region_ids)
    )
    result = result[0]
    
    # We need the candidates, indexed by order id but sorted as above to know where the orders are located
    delta_base = candidates.reset_index().set_index('order_id')
    
    return result, delta_base

# Encoding of an order delta that can be applied to the base order tensor retrieved from the dataframe
def encode_delta(delta_base, state):
    if not state.updated_orders:
        return lambda x: x
    
    # Compute the indices of orders requiring updates
    indices = [
        delta_base.index.get_loc(oid) for oid, _ in state.updated_orders
    ]
    
    # Encode the updated orders
    orders = list(delta_base.iloc[indices].reset_index().itertuples())
    updates = [
        encode_order(o) for o in orders
    ]
    
    # Create a function that applies the update to an encoded tensor
    indices = tf.constant([[i] for i in indices], dtype = tf.int32)
    updates = tf.constant(updates, dtype = tf.float32)
    
    def apply(x):        
        return tf.ragged.map_flat_values(
            tf.tensor_scatter_nd_update,
            x, indices, updates
        )
    
    return apply

In [80]:
# Test for encoder
encoded, delta_base = encode_orders(orders)

s = MutableState()
s.updated_orders = {5581552764: 5}
s = State(s)

for i in trange(0, 1000):
    encode_delta(delta_base, s)

Encoding orders & transferring to GPU


In [91]:
# --- Global embedding of data ---

# Helper layer that holds the encoded universe and can map order tensors to their full representation
class Universe(tf.keras.layers.Layer):
    def __init__(self, d_system_notes = 8, d_type_notes = 8):
        super(Universe, self).__init__()
        
        self.system_data_const = tf.constant([encode_system(system) for system in tqdm(system_ids, desc = 'Encoding systems')], dtype = tf.float32)
        self.system_data_var   = tf.Variable(np.zeros([len(system_ids), d_system_notes]), dtype = tf.float32)

        self.type_data_const = tf.constant([encode_type(type_id) for type_id in tqdm(types, desc = 'Encoding types')], dtype = tf.float32)
        self.type_data_var   = tf.Variable(np.zeros([len(types), d_type_notes]), dtype = tf.float32)

    def system_data(self):
        with tf.name_scope('Universe', reuse=True):
            with tf.name_scope('SystemData', reuse = True):
                return tf.concat([self.system_data_const, self.system_data_var], axis = -1)

    def type_data(self):
        with tf.name_scope('Universe', reuse = True):
            with tf.name_scope('TypeData', reuse = True):
                return tf.concat([self.type_data_const, self.type_data_var], axis = -1)

    def embed_order(self, order_tensor):
        with tf.name_scope('Universe', reuse = True):
            with tf.name_scope('OrderEmbedding', reuse = True):
                sys_idx = tf.cast(order_tensor[...,0], tf.int32)
                typ_idx = tf.cast(order_tensor[...,1], tf.int32)

                return tf.concat(
                    [
                        tf.gather(self.system_data(), sys_idx),
                        tf.gather(self.type_data(),   typ_idx),
                        order_tensor[...,2:]
                    ],
                    axis = -1
                )
    

In [92]:
# Attention layers

def full_attention(keys, values, queries, mask_kv = None, mask_q = None):
    """
        keys    - batch_shape + [n_sequence1, dim_k] tensor holding query keys
        values  - batch_shape + [n_sequence1, dim_v] tensor holding query results
        queries - batch_shape + [n_sequence2, dim_k] tensor holding queries
        mask_kv - batch_shape + [n_sequence1] tensor holding the mask for the keys / values (0 if)
        mask_q  - batch_shape + [n_sequence2] tensor holding the mask for the 

        Returns batch_shape + [n_sequence2, dim_v] tensor holding result
    """
    with tf.name_scope('scaled_dot_product_attention'):
        # Compute the [..., n_sequence2, n_sequence1] matrix of weights
        weights = tf.linalg.matmul(queries, keys, transpose_b = True)
        
        # Normalize against the multiplied dimension dimension
        dk = tf.cast(tf.shape(keys)[-1], tf.float32)
        weights = weights / tf.math.sqrt(dk)
        
        if mask_kv is not None or mask_q is not None:
            mask_kv = mask_kv if mask_kv is not None else [True]
            mask_q  = mask_q  if mask_q  is not None else [True]
            
            mask = tf.math.logical_and(
                tf.expand_dims(mask_kv, axis = -2),
                tf.expand_dims(mask_q , axis = -1)
            )
            
            weights = tf.where(mask, weights, -1e9)
        
        # Compute softmax over the sequence1 dimension
        weights = tf.nn.softmax(weights, axis = -1)
        
        # Apply weights to values
        result = tf.linalg.matmul(weights, values)
        
        return result


def reduced_attention(keys, values, queries, mask_kv = None, mask_q = None):
    """
        keys    - batch_shape + [n_sequence1, dim_k] tensor holding query keys
        values  - batch_shape + [n_sequence1, dim_v] tensor holding query results
        queries - batch_shape + [n_sequence2, dim_k] tensor holding queries

        Returns batch_shape + [n_sequence2, dim_v] tensor holding result
    """
    with tf.name_scope('reduced_scaled_dot_product_attention'):
        keys = tf.where(mask_q, keys, 0)
        
        # Compute the [..., dim_k, dim_v] matrix of weights
        weights = tf.linalg.matmul(keys, values, transpose_a = True)
        
        # Normalize against the key dimension
        if mask_kv is not None:
            dn = tf.reduce_sum(tf.cast(mask_kv, tf.float32), axis = -1)
            
            # We need to expand from a batch_shape to a batch_shape + [1, 1] array
            # to maintain correct broadcasting
            dn = tf.expand_dims(dn, axis = -1)
            dn = tf.expand_dims(dn, axis = -1)
        else:
            dn = tf.cast(tf.shape(keys)[-2], tf.float32)
            
        weights = weights / tf.math.sqrt(dk)
        
        # Compute softmax over the dim_k dimension
        weights = tf.nn.softmax(weights, axis = -2)
        
        # Apply weights to values
        result = tf.linalg.matmul(queries, weights)
        
        return result

class MultiHeadedAttention(tf.keras.layers.Layer):
    def __init__(self, d_out, d_model, num_heads, f = reduced_attention, skip = True):
        super(MultiHeadedAttention, self).__init__()
        
        self.d_model = d_model
        self.num_heads = num_heads
    
        self.wk = tf.keras.layers.Dense(num_heads * d_model, name = 'Wk')
        self.wv = tf.keras.layers.Dense(num_heads * d_model, name = 'Wv')
        self.wq = tf.keras.layers.Dense(num_heads * d_model, name = 'Wq')
        
        self.wout = tf.keras.layers.Dense(d_out, name = 'Wout')
        
        self.f = f
        self.skip = skip
        
    def _split_heads(self, x):
        with tf.name_scope('split_heads'):
            # Ensure static shape
            #x.set_shape(tf.TensorShape([None, None, num_heads * d_model]))
            #static_shape = x.get_shape();

            # Dynamic reshape
            batch_size = tf.shape(x)[0]
            seq_len = tf.shape(x)[1]

            x = tf.reshape(x, [batch_size, seq_len, self.num_heads, self.d_model])

            # Static reshape
            # x.set_shape(static_shape[0:2] + tf.TensorShape([num_heads, d_model]))

            x = tf.transpose(x, [0, 2, 1, 3])
        return x
    
    def _merge_heads(self, x):
        with tf.name_scope('merge_heads'):
            batch_size = tf.shape(x)[0]
            seq_len    = tf.shape(x)[2]

            x = tf.transpose(x, [0, 2, 1, 3])
            x = tf.reshape(x, [batch_size, seq_len, self.num_heads * self.d_model])
            return x
        
    def call(self, kvq, mask = None):
        if isinstance(kvq, tuple):
            k, v, q = kvq
            
            if mask is not None:
                kv_mask = mask[0]
                q_mask  = mask[2]
        else:
            k = kvq
            v = kvq
            q = kvq
            kv_mask = mask
            q_mask  = mask

        k = self.wk(k)
        v = self.wv(v)
        q = self.wq(q)
        
        k = self._split_heads(k)
        v = self._split_heads(v)
        q = self._split_heads(q)
        
        result = self.f(k, v, q)
        result = self._merge_heads(result)
        
        result = self.wout(result)
        
        if self.skip:
            result += q
        
        return result
    
    def compute_mask(self, kvq, mask = None):
        if isinstance(kvq, tuple) and mask is not None:
            return mask[2]
        
        return mask

In [93]:
# --- Wraps another callable as a layer that accepts packed tensors ---

class StackedTogether(tf.keras.layers.Layer):
    def __init__(self, wrapped, axis = -2, mask_axis = None):
        super(StackedTogether, self).__init__()
        
        self._supports_ragged_inputs = True
        
        self.wrapped = wrapped
        self.axis = axis
        self.mask_axis = None
    
    def call(inputs, mask = None, *args, **kwargs):
        unpacked = tf.nest.flatten(inputs)
        sizes = [tf.shape(x)[self.axis] for x in unpacked]
        
        if mask is not None:
            assert self.mask_axis is not None, 'No axis for mask stacking specified'
            mask = tf.stack(
                tf.nest.flatten(mask),
                axis = self.mask_axis
            )
            kwargs['mask'] = mask
        
        stacked = tf.stack(unpacked, axis = self.axis)
        processed = self.wrapped(stacked, *args, **kwargs)
        unstacked = tf.split(processed, sizes)
        
        packed = tf.nest.pack_sequence_as(inputs, unstacked)
        
        return packed

In [95]:
# ------------------ Model class -----------------------------------------

class Model(tf.keras.Model):
    def __init__(self, n_layers = 3, bandwidth = 128, n_heads = 16, head_dimension = 8):
        super(Model, self).__init__()
        
        # Variable inputs feeding into the network
        self.universe    = Universe()
        self.action_feed = tf.Variable(np.zeros([bandwidth]), dtype = tf.float32)
        
        # Input embeddings
        self.embeddings = {
            k : tf.keras.layers.Dense(bandwidth)
            for k in ['systems', 'types', 'orders']
        }
        
        substack = tf.keras.Sequential([
            MultiHeadedAttention(bandwidth, n_heads, head_dimension)
            for i in range(0, n_layers)
        ])
        
        self.substack = StackedTogether(substack, axis = -2, mask_axis = -1)
        
        self.exit_move = tf.keras.layers.Dense(1)
        self.exit_buysell = tf.keras.layers.Dense(6)
        self.exit_action = tf.keras.layers.Dense(3)
        
    # TODO: Masking
    def call(self, input):
        orders = input['orders']
        
        data = {
            'systems' : self.universe.system_data(),
            'types' : self.universe.type_data(),
            'orders' : self.universe.order_data(orders)
        }
        
        data = {
            k : self.embeddings[k](v)
            for k,v in data.items()
        }
        
        data['actions'] = self.action_feed
        
        data = self.substack(data)
        
        action_params = self.exit_action(data['actions'])
        move_params   = self.exit_move(data['systems'])
        buysell_params = self.exit_buysell(data['types'])
        
        return action_params, move_params, buysell_params[...,:3], buysell_params[...,3:]

m = Model()

AttributeError: module 'tensorflow_core.keras' has no attribute 'Dense'

In [197]:
def sample_from(state, action_params, move_params, buy_params, sell_params):
    Joint = tfp.distributions.JointDistributionCoroutine
    
    def model():
        action_id = yield Joint.Root(tfp.distributions.Categorical(logits = action_params))
        
        if action_id == 0:
            target = yield tfp.distributions.Categorical(logits = move_params)
            target = system_ids[target.numpy()]
            
            return warp_to_system(target)
        
        if action_id == 1:
            params = buy_params
        elif action_id == 2:
            params = sell_params
            
        idx = yield tfp.distributions.Categorical(logits = params[...,0])
        amount = yield tfp.distributions.Normal(loc = params[...,1], scale = params[...,2])[0]
        
        if action_id == 1:
            return buy(type_ids[idx].numpy(), amount.numpy())
        elif action_id == 2:
            return sell([type_id for type_id, _ in state.cargo][idx.numpy()], amount.numpy())
        
        assert False
    
    dist = Joint(model)
    
    sample = dist.sample()
    log_p = dist.log_prob(sample)
        
    # Interpret sample
    generator = model()
    try:
        next(generator)
        for s in sample:
            generator.send(s)
    except StopIteration as s:
        return s.value, log_p

#def sample_chain(states, length, model, history):
#    log_p_tot = tf.constant(np.zeros([len(starts)], dtype = tf.float32))
#    
#    for i in range(0, length):
#        # Encode states
#        state_data = encode_states(states)
#        
#        control, history = model(state_data, history)
#        
#        actions, log_p = pick_actions(starts, *control)
#        
#        states = [action(state) for action, state in zip(actions, states)]
#        log_p_tot += log_p
#    
#    return states, log_p_tot

test_state = MutableState()
test_state.cargo = {
    34: (10, 0) # 10 units of Tritanium
}
test_state.volume_limit = 8900
test_state.time_left = 100

test_state.system = root_id # in jita

state = State(test_state)
action_params = tf.constant([0, -1000, -1], dtype = tf.float32)
system_logits = tf.constant([
        0 if sid in landmarks else -1000
        for sid in system_ids
    ],
    dtype = tf.float32
)

buy_params = tf.zeros([len(type_ids), 3], dtype = tf.float32)
sell_params = tf.constant([[0, 5, 0]], dtype = tf.float32)

print(state)

log_p_tot = 0

with tqdm(total = 10, desc = 'Successful') as bar:
    for i in trange(0, 10, desc = 'Iterations'):
        chosen_action, log_p = sample_from(state, action_params, system_logits, buy_params, sell_params)
        log_p_tot += log_p
        print(chosen_action)
        try:
            state = chosen_action(state)
            bar.update(1)
            print(state)
        except AssertionError as e:
            print('Failed')


State:
    Time left: 100
    
    System:  30000142
    Station: None
    
    Collateral: 0 / 0 (0 left)
    Cargo:      0.1000000000003638 / 8900 (8899.9 left)
    
    Value:     0
        


Move to Rens

State:
    Time left: 73.0
    
    System:  30002510
    Station: None
    
    Collateral: 0 / 0 (0 left)
    Cargo:      0.1000000000003638 / 8900 (8899.9 left)
    
    Value:     0
        
Sell 5.0 of Tritanium

State:
    Time left: 73.0
    
    System:  30002510
    Station: None
    
    Collateral: 0 / 0 (0 left)
    Cargo:      0.1000000000003638 / 8900 (8899.9 left)
    
    Value:     0
        
Move to Hek

State:
    Time left: 67.0
    
    System:  30002053
    Station: None
    
    Collateral: 0 / 0 (0 left)
    Cargo:      0.1000000000003638 / 8900 (8899.9 left)
    
    Value:     0
        
Sell 5.0 of Tritanium

State:
    Time left: 67.0
    
    System:  30002053
    Station: None
    
    Collateral: 0 / 0 (0 left)
    Cargo:      0.1000000000003638 / 8900 (8899.9 left)
    
    Value:     0
        
Sell 5.0 of Tritanium

State:
    Time left: 67.0
    
    System:  30002053
    Station: None
    
    Collateral: 0 / 0 (0 left)
    Cargo:      

In [130]:
test_state = MutableState()

idgen = iter(delta_base[0])
idgen = itertools.islice(idgen, 300)

for i in idgen:
    test_state.updated_orders[i] = 10
    
#delta = encode_delta(delta_base, State(test_state))

#for i in trange(0, 1000):
#    delta = encode_delta(delta_base, State(test_state))
#    encoded_2 = delta(encoded)

encoded_2 = [
    encode_delta(delta_base, State(test_state)) for i in trange(0, 1000)
]

#diff = encoded_2 - encoded

#diff.flat_values[0:3]

In [132]:
encoded_3 = [delta(encoded) for delta in tqdm(encoded_2)]

In [9]:
rval = tfp.edward2.Categorical(logits = [0, 0, 0])

d:\programme\python 3.6.6\lib\site-packages\ipykernel_launcher.py:1: UserWarning: tfp.edward2 module is deprecated and will be removed on 2019-12-01. Use https://github.com/google/edward2 library instead.
  """Entry point for launching an IPython kernel.


In [57]:
m = Model(3)

@tf.function
def f(x):
    y = m(x)
    return y

import datetime
stamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = 'logs/func'
writer = tf.summary.create_file_writer(logdir)

tf.summary.trace_on(graph=True, profiler=True)
# Call only one tf.function when tracing.
z = f(encoded_orders)

with writer.as_default():
    tf.summary.trace_export(
        name="my_func_trace_2",
        step=0,
        profiler_outdir=logdir)


In [13]:
tf.summary.trace_off()

In [26]:
for o in orders.values():
    print(o[0].keys())
    break

dict_keys(['duration', 'is_buy_order', 'issued', 'location_id', 'min_volume', 'order_id', 'price', 'range', 'system_id', 'type_id', 'volume_remain', 'volume_total'])


In [ ]:
r = tf.ragged.constant([[[0]], [[2], [3]]])

In [ ]:
r.to_tensor()

In [ ]:
s = MutableState()

s.system = root_id
s.time_left = 40.0

ship = 'tayra'

if ship == 'charon':
    s.volume_limit = 1350000
    s.collateral_limit = 750000000
elif ship == 'tayra':
    s.volume_limit = 8395
    s.collateral_limit = 100000000

expanded = set([])

def heur(x):
    return x.value()

g, unexp = expand(State(s), heur, 30000)

heurmax = max(*[heur(n) for n in g])
print(heurmax)

In [ ]:
import cProfile

cProfile.run("expand(State(s), heur, 10000)")

In [ ]:
cycles = nx.simple_cycles(g);

for c in cycles:
    print(c)

In [ ]:
plt.figure(figsize=(20, 20))

nx.draw_networkx(
    g,
    labels = {n:systems[n.system]["name"] for n in g},
    node_size = [300 * heur(n) / heurmax for n in g],
    node_color = [heur(n) / heurmax for n in g],
    cmap = 'jet'
)

c = matplotlib.colorbar.ColorbarBase(plt.gca(), cmap='jet')

In [ ]:
vals = get_values(State(s), g, unexp, heur)

In [ ]:
len(g)

In [ ]:
plt.figure(figsize=(20, 20))

valmax = max(*vals.values())

subg = g.subgraph([n for n in g if vals[n] == valmax])

nx.draw_networkx(
    subg,
    labels = {n:systems[n.system]["name"] for n in subg},
    node_size = [30 * n.time_left for n in subg],
    node_color = [300 * heur(n) / heurmax for n in subg],
    alpha = 1.0,
    cmap = 'jet'
)

plt.show()

for n in nx.dfs_preorder_nodes(g, State(s)):
    if vals[n] < valmax:
        continue
    
    print(n)

In [ ]:
for n in nx.dfs_preorder_nodes(g, State(s)):
    if not n.loaded:
        continue
    
    print(n)

In [ ]:
plt.hist([heur(n) for n in g]);

In [ ]:
vals

In [ ]:
systems[root_id]

In [20]:
import pyswagger

dir(pyswagger.primitives.Model)

['__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'apply_with',
 'cleanup',
 'clear',
 'copy',
 'fromkeys',
 'get',
 'items',
 'keys',
 'pop',
 'popitem',
 'setdefault',
 'update',
 'values']

In [26]:
with open('orders.dat', 'wb') as f:
    pickle.dump(orders, f)

KeyError: '__getstate__'

In [23]:
type(orders)

dict

In [30]:
df

KeyError: '__next__'

KeyError: '__next__'

In [77]:
orders[
        (orders.system_id.isin(system_ids)) &
        (orders.type_id.isin(type_ids))
    ].set_index(['region_id', 'type_id']).sort_index()

duration  is_buy_order                     issued  \
region_id type_id                                                      
10000001  18             30         False  2020-02-12T08:46:44+00:00   
          20             90          True  2020-01-15T10:42:07+00:00   
          36             90          True  2020-02-21T14:48:14+00:00   
          38             90          True  2020-02-19T23:15:17+00:00   
          39             90          True  2020-02-22T10:34:55+00:00   
...                     ...           ...                        ...   
10000068  54162          90          True  2020-02-13T10:14:43+00:00   
          54163          90          True  2020-02-13T10:13:09+00:00   
          54164          90          True  2020-02-13T10:12:09+00:00   
          54165          90          True  2020-02-13T10:11:54+00:00   
          54166          90          True  2020-02-19T07:22:14+00:00   

                     location_id  min_volume   price        range  system_id  \
region_id type_id                                                              
10000001  18            60013009           1  128.00       region   30000087   
          20            60007573           1  171.00  solarsystem   30000031   
          36            60012124           1   24.00           40   30000005   
          38       1026200541472           1  203.00       region   30000008   
          39            60012154           1  530.00      station   30000033   
...                          ...         ...     ...          ...        ...   
10000068  54162         60010399           1    1.00       region   30005301   
          54163         60010399           1    1.00       region   30005301   
          54164         60010399           1    1.00       region   30005301   
          54165         60010399           1    1.00       region   30005301   
          54166         60010399           1    1.82       region   30005301   

                   volume_remain  volume_total  
region_id type_id                               
10000001  18                1768          1768  
          20               71335        800000  
          36              447307       1344433  
          38              952742       1000000  
          39               49999         50000  
...                          ...           ...  
10000068  54162              998          1000  
          54163              985          1000  
          54164              998          1000  
          54165              989          1000  
          54166              987          1000  

[123510 rows x 10 columns]